In [1]:
import pandas as pd

In [2]:
adp = pd.read_csv('adp_2324.csv')

In [3]:
#adp['team'] = adp['player'].apply(lambda x: x.split('\xa0')[-1])

In [4]:
#adp['player'] = adp['player'].apply(lambda x: ' '.join(x.split('\xa0')[:-1]).upper())

In [3]:
adp['player'] = adp['player'].apply(lambda x: x \
        .replace('ALEKSANDER BARKOV JR.', 'ALEKSANDER BARKOV') \
        .replace('MITCH MARNER', 'MITCHELL MARNER') \
        .replace('MACKENZIE BLACKWOOD-DUP', 'MACKENZIE BLACKWOOD') \
        .replace('MAXIME COMTOIS', 'MAX COMTOIS')) \
        .replace('JT MILLER', 'J.T. MILLER') \
        .replace('TJ OSHIE', 'T.J. OSHIE') \
        .replace('JT COMPHER', 'J.T. COMPHER') \
        .replace('TIM STUETZLE', 'TIM STUTZLE') \
        .replace('NICK PAUL', 'NICHOLAS PAUL')

In [4]:
#adp = adp[['yahoo', 'player']]
adp.columns = ['ADP', 'Name']

In [5]:
adp['ADP'] = pd.to_numeric(adp['ADP'], errors='coerce')

In [20]:
dfs = {}
ss = ['goalies', 'peripherals', 'pp', 'skaters']
ys = ['2223', '2122', '2021']
for s in ss:
    for y in ys:
        df = pd.read_csv(f'{s}_{y}.csv')
        df['Name'][(df['Name']=='SEBASTIAN AHO') & (df['Team']=='NYI')] = 'SEBASTIAN AHO NYI'
        dfs[(s,y)] = df

/tmp/ipykernel_197/202203092.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'][(df['Name']=='SEBASTIAN AHO') & (df['Team']=='NYI')] = 'SEBASTIAN AHO NYI'
/tmp/ipykernel_197/202203092.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'][(df['Name']=='SEBASTIAN AHO') & (df['Team']=='NYI')] = 'SEBASTIAN AHO NYI'
/tmp/ipykernel_197/202203092.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'][(df['Name']==

In [21]:
for y in ys:
    df = dfs[('skaters',y)] \
        .merge(dfs[('peripherals',y)], how='outer', on='Name') \
        .merge(dfs[('pp',y)], how='outer', on='Name')
    df = df[['Name', 'Pos_x', 'Age_x', 'Team_x', 'GP_x', 'G_x',
            'A_x', 'PTS', 'GWG', 'TOI_x', 'SOG_x', 'Yrs', 'Hits',
            'Blocks', 'PPG', 'PPP']]
    df.columns = [x.replace('_x', '') for x in df.columns]
    dfs[('merged',y)] = df
    dfs[('goalies',y)] = dfs[('goalies',y)][['Name', 'Age', 
            'Team', 'GP', 'W', 'SO', 'SV', 'SV%', 'Yrs']]

In [22]:
joins = ['Name']
for s in ['goalies', 'merged']:
    dfs[(s,'2021')].columns = [f'{x}_2021' if x not in joins else x for x in dfs[(s,'2021')].columns]
    dfs[s,'merged'] = dfs[(s,'2223')] \
        .merge(dfs[(s,'2122')], how='left', on=joins, suffixes=('_2223','_2122')) \
        .merge(dfs[(s,'2021')], how='left', on=joins)

In [24]:
merge = pd.concat([dfs['goalies','merged'],dfs['merged','merged']])

In [25]:
merge = merge.merge(adp, on='Name', how='outer')

In [26]:
merge['Pos'] = merge['Pos_2223'].apply(lambda x: 'D' if x == 'D' else 'F' if x in ['C', 'L', 'R'] else 'G')

In [27]:
pl = pd.read_csv('placeholders.csv')
merge = pd.concat([merge, pl])

In [44]:
merge['Age'] = merge['Age_2223']
merge = merge.drop(columns=['Age_2223', 'Age_2122', 'Age_2021'])

In [56]:
merge.to_csv('data.csv', index=False)

In [57]:
print(merge[merge['Age'].isna()]['Name'])

0    PLACEHOLDER_G
1    PLACEHOLDER_F
2    PLACEHOLDER_D
Name: Name, dtype: object


In [55]:
merge.loc[merge['Name'] == 'CONNOR BEDARD', 'Age'] = 18
merge.loc[merge['Name'] == 'SEAN COUTURIER', 'Age'] = 30
merge.loc[merge['Name'] == 'LOGAN COOLEY', 'Age'] = 19
merge.loc[merge['Name'] == 'ADAM FANTILLI', 'Age'] = 18
merge.loc[merge['Name'] == 'FABIAN LYSELL', 'Age'] = 20

In [31]:
print(len(merge))

1066


In [46]:
merge.columns

Index(['Name', 'Team_2223', 'GP_2223', 'W_2223', 'SO_2223', 'SV_2223',
       'SV%_2223', 'Yrs_2223', 'Team_2122', 'GP_2122', 'W_2122', 'SO_2122',
       'SV_2122', 'SV%_2122', 'Yrs_2122', 'Team_2021', 'GP_2021', 'W_2021',
       'SO_2021', 'SV_2021', 'SV%_2021', 'Yrs_2021', 'Pos_2223', 'G_2223',
       'A_2223', 'PTS_2223', 'GWG_2223', 'TOI_2223', 'SOG_2223', 'Hits_2223',
       'Blocks_2223', 'PPG_2223', 'PPP_2223', 'Pos_2122', 'G_2122', 'A_2122',
       'PTS_2122', 'GWG_2122', 'TOI_2122', 'SOG_2122', 'Hits_2122',
       'Blocks_2122', 'PPG_2122', 'PPP_2122', 'Pos_2021', 'G_2021', 'A_2021',
       'PTS_2021', 'GWG_2021', 'TOI_2021', 'SOG_2021', 'Hits_2021',
       'Blocks_2021', 'PPG_2021', 'PPP_2021', 'ADP', 'Pos', 'Age'],
      dtype='object')

In [35]:
merge[merge['Pos'] == 'G'][['GP_2223', 'SV%_2223', 'W_2223', 'SO_2223', 'SV_2223']].describe()

,GP_2223,SV%_2223,W_2223,SO_2223,SV_2223
count,108.000000,108.000000,108.000000,108.000000,108.000000
mean,26.157407,0.894231,12.314815,1.083333,692.472222
std,19.113122,0.090117,10.666261,1.454127,528.266314
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,0.892000,3.000000,0.000000,205.500000
50%,23.500000,0.902500,9.500000,0.500000,604.500000
75%,39.250000,0.915000,18.250000,2.000000,1048.250000
max,64.000000,1.000000,40.000000,6.000000,1928.000000


In [47]:
merge.dtypes

Name            object
Team_2223       object
GP_2223        float64
W_2223         float64
SO_2223        float64
SV_2223        float64
SV%_2223       float64
Yrs_2223       float64
Team_2122       object
GP_2122        float64
W_2122         float64
SO_2122        float64
SV_2122        float64
SV%_2122       float64
Yrs_2122       float64
Team_2021       object
GP_2021        float64
W_2021         float64
SO_2021        float64
SV_2021        float64
SV%_2021       float64
Yrs_2021       float64
Pos_2223        object
G_2223         float64
A_2223         float64
PTS_2223       float64
GWG_2223       float64
TOI_2223        object
SOG_2223       float64
Hits_2223      float64
Blocks_2223    float64
PPG_2223       float64
PPP_2223       float64
Pos_2122        object
G_2122         float64
A_2122         float64
PTS_2122       float64
GWG_2122       float64
TOI_2122        object
SOG_2122       float64
Hits_2122      float64
Blocks_2122    float64
PPG_2122       float64
PPP_2122   